### Import Libraries

In [1]:
import os

os.chdir('../scripts') # cd into scripts dir

from scan import *
from prep import *
from subset import *
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression

os.chdir('../') # cd back to parent dir
home_dir = "compas/" # set home dir for reading and storing data

# Read recipe inputs
compas_prep_df = pd.read_csv(home_dir + "datasets/compas_prep.csv")
compas_prep_df

,Unnamed: 0,sex,race,prior_offenses,under_25,charge_degree,outcomes,compas_risk_score,proba_compas,proba_lr
0,0,Male,Other,None,False,F,0,1,0.213889,0.232134
1,1,Male,African-American,None,False,F,1,3,0.376171,0.289774
2,2,Male,African-American,1 to 5,True,F,1,4,0.434330,0.668642
3,3,Male,African-American,1 to 5,True,F,0,8,0.683594,0.668642
4,4,Male,Other,1 to 5,False,F,0,1,0.213889,0.373653
...,...,...,...,...,...,...,...,...,...,...
7209,7209,Male,African-American,None,True,F,0,7,0.591216,0.505581
7210,7210,Male,African-American,None,True,F,0,3,0.376171,0.505581
7211,7211,Male,Other,None,False,F,0,1,0.213889,0.232134
7212,7212,Female,African-American,1 to 5,False,M,0,2,0.311371,0.335035


### Prepare Base Dataset for Mitigation

### Mitigation Thresholds

a) 0.45 threshold for everyone (base case)
b) 0.5 threshold for Black individuals, 0.45 for everyone else (increased threshold for Black defendants)
c) 0.45 threshold for Black individuals, 0.4 for everyone else (decreased threshold for non-Black defendants)
d) 0.5 threshold for Black individuals, 0.4 for everyone else (increased threshold for Black defendants and decreased threshold for non-Black defendants)

In [2]:
threshold_labels = ['a', 'b', 'c', 'd']
threshold_settings = {'a' : (0.45, 0.45),
                      'b' : (0.5, 0.45),
                      'c' : (0.45, 0.4),
                      'd' : (0.5, 0.4)}
for label in threshold_labels:
    setting = threshold_settings[label]
    compas_prep_df['threshold_' + label] = compas_prep_df['race'].apply(lambda x : setting[0] if x == 'African-American'
                                                                                    else setting[1])
compas_prep_df

,Unnamed: 0,sex,race,prior_offenses,under_25,charge_degree,outcomes,compas_risk_score,proba_compas,proba_lr,threshold_a,threshold_b,threshold_c,threshold_d
0,0,Male,Other,None,False,F,0,1,0.213889,0.232134,0.45,0.45,0.40,0.4
1,1,Male,African-American,None,False,F,1,3,0.376171,0.289774,0.45,0.50,0.45,0.5
2,2,Male,African-American,1 to 5,True,F,1,4,0.434330,0.668642,0.45,0.50,0.45,0.5
3,3,Male,African-American,1 to 5,True,F,0,8,0.683594,0.668642,0.45,0.50,0.45,0.5
4,4,Male,Other,1 to 5,False,F,0,1,0.213889,0.373653,0.45,0.45,0.40,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,7209,Male,African-American,None,True,F,0,7,0.591216,0.505581,0.45,0.50,0.45,0.5
7210,7210,Male,African-American,None,True,F,0,3,0.376171,0.505581,0.45,0.50,0.45,0.5
7211,7211,Male,Other,None,False,F,0,1,0.213889,0.232134,0.45,0.45,0.40,0.4
7212,7212,Female,African-American,1 to 5,False,M,0,2,0.311371,0.335035,0.45,0.50,0.45,0.5


In [3]:
# specify probability and outcomes columns
PROBA_COL = 'proba_compas'
OUTCOMES_COL = 'outcomes'
FEATURES = ['sex', 'race', 'under_25', 'charge_degree', 'prior_offenses']
pd.options.mode.chained_assignment = None  # suppress warnings on chained assignment

# filter for negative and positive outcomes
negatives_df = compas_prep_df.loc[compas_prep_df[OUTCOMES_COL] == 0]
positives_df = compas_prep_df.loc[compas_prep_df[OUTCOMES_COL] == 1]

In [4]:
# report FPR and TPR of African-Americans and non African-Americans
for label in threshold_labels:
    negative_metrics_df = generate_metrics(negatives_df, PROBA_COL, OUTCOMES_COL,
                                           'threshold_' + label, constant_threshold=False)
    positive_metrics_df = generate_metrics(positives_df, PROBA_COL, OUTCOMES_COL,
                                           'threshold_' + label, constant_threshold=False)
    negative_metrics_aa_df = negative_metrics_df.loc[negative_metrics_df['race'] == 'African-American']
    negative_metrics_other_df = negative_metrics_df.loc[negative_metrics_df['race'] != 'African-American']
    positive_metrics_aa_df = positive_metrics_df.loc[positive_metrics_df['race'] == 'African-American']
    positive_metrics_other_df = positive_metrics_df.loc[positive_metrics_df['race'] != 'African-American']
    print("FPR of African-Americans", negative_metrics_aa_df['positives'].sum() / len(negative_metrics_aa_df))
    print("FPR of non African-Americans", negative_metrics_other_df['positives'].sum() / len(negative_metrics_other_df))
    print("TPR of African-Americans", positive_metrics_aa_df['positives'].sum() / len(positive_metrics_aa_df))
    print("TPR of non African-Americans", positive_metrics_other_df['positives'].sum() / len(positive_metrics_other_df))

FPR of African-Americans 0.44846796657381616
FPR of non African-Americans 0.22001845018450183
TPR of African-Americans 0.7201472908995266
TPR of non African-Americans 0.49333333333333335
FPR of African-Americans 0.34317548746518106
FPR of non African-Americans 0.22001845018450183
TPR of African-Americans 0.6275644397685429
TPR of non African-Americans 0.49333333333333335
FPR of African-Americans 0.44846796657381616
FPR of non African-Americans 0.3247232472324723
TPR of African-Americans 0.7201472908995266
TPR of non African-Americans 0.6096296296296296
FPR of African-Americans 0.34317548746518106
FPR of non African-Americans 0.3247232472324723
TPR of African-Americans 0.6275644397685429
TPR of non African-Americans 0.6096296296296296


### Mitigation Results

Set lambda vales to test for each threshold case. Only need to score subgroup S in each case, no need to run the full IJDI-Scan.

In [5]:
lambda_vals = [0, 0.3, 1, 3, 10]
race_subset = {'race': ['African-American']}
n_iters = 1

# specify parameters for generating metrics and IJDI scan
proba_confusion_col = 'proba_compas'
proba_ijdi_col = 'proba_lr'
outcomes_col = 'outcomes'
features = ['sex', 'race', 'under_25', 'charge_degree', 'prior_offenses']

pd.options.mode.chained_assignment = None  # suppress warnings on chained assignment

# set random seed
np.random.seed(100)

In [6]:
sim_data = []

for label in threshold_labels:

    setting = threshold_settings[label]
    threshold = 'threshold_' + label
    print("Thresholds: African-Americans {}, All Other Races {}".format(setting[0], setting[1]))

    for i in range(n_iters): # run n iterations for each k value

        print("Iteration", i+1, "of", n_iters)

        # filter for negative and positive outcomes
        negatives_df = compas_prep_df.loc[compas_prep_df[OUTCOMES_COL] == 0]
        positives_df = compas_prep_df.loc[compas_prep_df[OUTCOMES_COL] == 1]

        for lambda_param in lambda_vals: # run IJDI scan for various lambda values

            print("Lambda =", lambda_param)

            # Run Negative and Positive IJDI Scan. Make sure to pass in copy because data may be modified by the function!
            negative_score = score_current_subset_ijdi(negatives_df.copy(deep=True), features, proba_confusion_col, proba_ijdi_col, outcomes_col,
                                                       threshold, lambda_param, race_subset, constant_threshold=False, verbose=True)

            print("Score for Negative IJDI-Scan:", negative_score)

            positive_score = score_current_subset_ijdi(positives_df.copy(deep=True), features, proba_confusion_col, proba_ijdi_col, outcomes_col,
                                                       threshold, lambda_param, race_subset, constant_threshold=False, verbose=True)

            print("Score Positive IJDI-Scan:", positive_score)

            # append data
            negative_row = [setting[0], setting[1], lambda_param, negative_score, 'negative']
            positive_row = [setting[0], setting[1], lambda_param, positive_score, 'positive']
            sim_data.append(negative_row)
            sim_data.append(positive_row)

            print(negative_row)
            print(positive_row)

            print("\n----------------------------------------------------\n")

Thresholds: African-Americans 0.45, All Other Races 0.45
Iteration 1 of 1
Lambda = 0
Current Score: 60.66139161495723
First Iteration
Average p_delta: 0.06005602252451366
Average p_censor: 0.0
      proba_lr     p_old     p_bar  p_bar_old   p_delta  positives_constprobs  \
3     0.668642  0.668642  0.397193   0.397193  0.271449              0.323492   
13    0.289774  0.289774  0.397193   0.397193 -0.107418              0.323492   
17    0.446022  0.446022  0.397193   0.397193  0.048829              0.323492   
40    0.599215  0.599215  0.397193   0.397193  0.202023              0.323492   
41    0.404766  0.404766  0.397193   0.397193  0.007573              0.323492   
...        ...       ...       ...        ...       ...                   ...   
7204  0.404766  0.404766  0.397193   0.397193  0.007573              0.323492   
7208  0.505581  0.505581  0.397193   0.397193  0.108389              0.323492   
7209  0.505581  0.505581  0.397193   0.397193  0.108389              0.323492 

In [7]:
columns = ['threshold_aa', 'threshold_other', 'lambda', 'score', 'scan_type']
mit_result_df = pd.DataFrame(sim_data, columns=columns)
mit_result_df

,threshold_aa,threshold_other,lambda,score,scan_type
0,0.45,0.45,0.0,60.661392,negative
1,0.45,0.45,0.0,37.457971,positive
2,0.45,0.45,0.3,44.188756,negative
3,0.45,0.45,0.3,26.598115,positive
4,0.45,0.45,1.0,17.380486,negative
5,0.45,0.45,1.0,8.649653,positive
6,0.45,0.45,3.0,0.000000,negative
7,0.45,0.45,3.0,0.000000,positive
8,0.45,0.45,10.0,0.000000,negative
9,0.45,0.45,10.0,0.000000,positive


In [8]:
# Write recipe outputs
mit_result_df.to_csv(home_dir + "datasets/compas_mit_1.csv")